In [2]:
import gurobipy as gp
from gurobipy import GRB

# Problem 1

Contois Carpets is a small manufacturer of carpeting for home and office installations. Production
capacity, estimated demand, production cost and inventory holding cost are shown in table 1. Contois
wants to determine how many square meters of carpeting to produce each quarter to minimize the
total production and inventory cost for the four-quarter period.
Contois has control over the production quantity, and therefore also the estimated inventory,
each quarter. We can model the problem by using both the production and inventory quantities
as decision variables, and linking them through constraints. The decision variables x_i describe
production quantities and s_i the inventory quantities each quarter.


<img src="problem1.png">

In [4]:
try:
    m = gp.Model("exercise1") # Setting up the model
    
    # x = Amount of carpet produced in quarter
    x1 = m.addVar(vtype=GRB.CONTINUOUS, name="x1")
    x2 = m.addVar(vtype=GRB.CONTINUOUS, name="x2")
    x3 = m.addVar(vtype=GRB.CONTINUOUS, name="x3")
    x4 = m.addVar(vtype=GRB.CONTINUOUS, name="x4")
    
    # s = Amount of carpet in inventory in quarter
    s1 = m.addVar(vtype=GRB.CONTINUOUS, name="s1")
    s2 = m.addVar(vtype=GRB.CONTINUOUS, name="s2")
    s3 = m.addVar(vtype=GRB.CONTINUOUS, name="s3")
    s4 = m.addVar(vtype=GRB.CONTINUOUS, name="s4")
    
    # Objective function
    m.setObjective(2*x1 + 5*x2 + 3*x3 + 3*x4 + 0.25*(s1 + s2 + s3 + s4), GRB.MINIMIZE)
    
    # Defining the constraints
    m.addConstr(x1 <= 600, "c0" )
    m.addConstr(x2 <= 300, "c1" )
    m.addConstr(x3 <= 500, "c2" )
    m.addConstr(x4 <= 400, "c3" )
    
    m.addConstr(x1 - s1 == 400, "c4" )
    m.addConstr(x2 + s1 - s2 == 500, "c5" )
    m.addConstr(x3 + s2 - s3 == 400, "c6" )
    m.addConstr(x4 + s3 - s4 == 400, "c7" )
    
    m.optimize() # Optimizing the model

    # Printing the results
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)


except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')   
    
    

Academic license - for non-commercial use only - expires 2021-05-07
Using license file C:\Users\User\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 8 columns and 15 nonzeros
Model fingerprint: 0x9d73936f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve removed 8 rows and 8 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  5.150000000e+03
x1 600
x2 300
x3 400
x4 400
s1 200
s2 0
s3 0
s4 0
Obj: 5150


# Problem 2

Assume that there are two projects (A and B). The decision maker (DM) can invest in both of
these projects in two phases: initial investment of 1M for A, $2M for B in period 0 and additional
investment of 3M and 2M, respectively, in period 1. The initial budget for all investments is 9M.
Additionally, surplus cash can be deposited with risk-free interest rate of 8%.
The resulting cash flows from the projects depend on which of two possible scenarios realize in
each period, and the three decisions: z_0_p? : initial investment in project p (1 variable per project) and
z_1_p: continue project ? in scenario 8 (2 variables per project).
If in period 1 scenario i realizes, the in period two the scenarios can be either s_i_1 or s_i_2.
The outcomes of investments in different scenarios are depicted in figure 1. Note that the DM knows
the outcome of scenario in period 1 when making the second investment (continue or not) decision.
If the project is not continued, then the it will produce no income.

<img src="problem2.png">

<img src="problem2_equation.png">

In [6]:
try:
    m = gp.Model("exercise2") # Setting up the model
    
    # Cash flow
    r0 = m.addVar(vtype=GRB.CONTINUOUS, name="r0")
    r1 = m.addVar(vtype=GRB.CONTINUOUS, name="r1")
    r2 = m.addVar(vtype=GRB.CONTINUOUS, name="r2")
    r11 = m.addVar(vtype=GRB.CONTINUOUS, name="r11")
    r12 = m.addVar(vtype=GRB.CONTINUOUS, name="r12")
    r22 = m.addVar(vtype=GRB.CONTINUOUS, name="r22")
    r21 = m.addVar(vtype=GRB.CONTINUOUS, name="r21")
    
    # Binary decision
    z0A = m.addVar(vtype=GRB.BINARY, name="z0A")
    z1A = m.addVar(vtype=GRB.BINARY, name="z1A")
    z2A = m.addVar(vtype=GRB.BINARY, name="z2A")
    z0B = m.addVar(vtype=GRB.BINARY, name="z0B")
    z1B = m.addVar(vtype=GRB.BINARY, name="z1B")
    z2B = m.addVar(vtype=GRB.BINARY, name="z2B")
    
    # Given probabilities
    p11 = 0.5 * 0.3
    p21 = 0.5 * 0.4
    p12 = 0.5 * 0.7
    p22 = 0.5 * 0.6
    
    # Objective function
    m.setObjective(p11*r11 + p21*r21 + p12*r12 + p22*r22, GRB.MAXIMIZE)
    
    ## Defining the constraints
    # Time = 0
    m.addConstr(9 - z0A - 2*z0B == r0, "c0" ) 
    
    # Time = 1
    m.addConstr(1.08*r0 - 3*z1A - 2*z1B == r1, "c1" )
    m.addConstr(1.08*r0 - 3*z2A - 2*z2B == r2, "c2" )
    
    # Time = 2
    m.addConstr(1.08*r1 + 20*z1A + 2.5*z1B == r11, "c3" )
    m.addConstr(1.08*r1 + 10*z1A + z1B == r12, "c4" )
    m.addConstr(1.08*r2 + 5*z2A + 25*z2B == r21, "c5" )
    m.addConstr(1.08*r2 + 10*z2B == r22, "c6" )

    # Decision constraints
    m.addConstr(z0A >= z1A, "c7" )
    m.addConstr(z0A >= z2A, "c8" )
    m.addConstr(z0B >= z1B, "c9" )
    m.addConstr(z0B >= z2B, "c10" )

    
    m.optimize() # Optimizing the model

    # Printing the results
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)


except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')   

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 11 rows, 13 columns and 34 nonzeros
Model fingerprint: 0xf6c65373
Variable types: 7 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Presolve removed 11 rows and 13 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 18.7984 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.879840000000e+01, best bound 1.879840000000e+01, gap 0.0000%
r0 6
r1 3.48
r2 4.48
r11 23.7584
r12 13.7584
r22 14.8384
r21 29.8384
z0A 1
z1A 1
z2A 0
z0B 1
z1B 0
z2B 1
Obj: 18.7984
